# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [141]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [142]:
import tweepy
import math
import os.path
import pandas as pd
import json
import numpy as np
from random import shuffle
import itertools 

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [143]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano
#Conta: @gabrielpfm

#leitura do arquivo no formato JSON
#with open('auth.pass') as fp:    
 #   data = json.load(fp)

#Configurando a biblioteca. Não modificar
#auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
#auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [144]:
#Produto escolhido:
#produto = 'Nintendo'

#Quantidade mínima de mensagens capturadas:
#n = 750
#Quantidade mínima de mensagens para a base de treinamento:
t = 450

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [145]:
#Cria um objeto para a captura
#api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
#i = 1
#msgs = []
#for msg in tweepy.Cursor(api.search, q=produto, lang=lang,tweet_mode="extended").items():    
    #msgs.append(msg.full_text.lower())
 #   i += 1
 # #  if i > n:
   #     break

#Embaralhando as mensagens para reduzir um possível viés
#shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [146]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
#if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
   # writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
  #  dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
   # dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

  #  dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
  #  dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
  #  writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [147]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [148]:
Excel = pd.read_excel("Nintendo.xlsx", sheet_name="Treinamento")


In [149]:
#Transformando Excel em 3 textos divididos por categoria

trein_cat = []
QTD_CAT = 3
for i in range(0,QTD_CAT):
    trein_cat.append(" ".join(Excel[Excel["Classificação"]==i].Treinamento))


In [157]:
#Três'textões' após a limpeza

cat_clean = []
serie = []
for i in range(0,QTD_CAT):
    cat_clean.append(cleanup(trein_cat[i]))
    serie.append(pd.Series(cat_clean[i].split()))

In [183]:
tudo = ""
for cat in cat_clean:
    tudo += cat
serie_tudo = pd.Series(tudo.split())

In [159]:
freq_abs = []
freq_rel = []
for i in range(0,QTD_CAT):
    freq_abs.append(serie[i].value_counts())
    freq_rel.append(serie[i].value_counts(True))
freq_abs.append(serie_tudo.value_counts())
freq_rel.append(serie_tudo.value_counts(True))

In [160]:
Probs = []
for i in range(0,QTD_CAT):
    Probs.append(freq_abs[i].sum()/freq_abs_tudo.sum())


In [176]:
set_=[]
for i in range(0,QTD_CAT):
    set_.append(set(freq_rel[i].index))
x = itertools.combinations(range(0,QTD_CAT),2)
inter = []
for i in x:
    i0=i[0]
    i1=i[1]
    inter.append(set_[i0].intersection(set_[i1]))


In [178]:
def Classificador(tweet,freq_abs):
    categoria = 0
    maior_prob = 0
    Prob_Tweet_Categoria = {}
    Prob_Categoria_Tweet = {}
    for n in range(0,3):
        probabilidade = 1
        probtc = freq_abs[n][tweet.values]
        for palavra,quantidade in probtc.items():


            if np.isnan(quantidade) == True:
                laplace = 1/(len(freq_abs_tudo)+len(freq_abs[n]))
            else:
                laplace = quantidade+1/(len(freq_abs_tudo)+len(freq_abs[n]))
                
        probabilidade *= laplace
        
                
                
                
        
            
        
        Prob_Tweet_Categoria[n] = probabilidade
    for n in range(0,3):
        prtoct = Prob_Tweet_Categoria[n]*Probs[n]
        Prob_Categoria_Tweet[n] = prtoct
    for categorias, probabilidade in Prob_Categoria_Tweet.items():
        if probabilidade > maior_prob:
            maior_prob = probabilidade
            
            categoria = categorias
    
    return categoria
        


In [179]:
#Entrada do Tweet
def Limpa_Tweet(Excel_Lido,Coluna,freq_rel):
    indice = 0
    Excel_Lido.loc[:,"Classificação"] = 0
    for tweets in Excel_Lido.Teste:
        tweet_limpo = cleanup(tweets.lower())
        tweet = pd.Series(tweet_limpo.split())
        classificado = Classificador(tweet,freq_rel)
        Excel_Lido.loc[indice,"Classificação"] = classificado
        indice +=1
    return Excel_Lido
    

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [180]:
Excel_Teste = pd.read_excel("Nintendo.xlsx", sheet_name="Teste")

manual =[]
for classficacao in Excel_Teste["Classificação"]:
    manual.append(classficacao)


teste = "Teste"
Excel_Classificado = Limpa_Tweet(Excel_Teste,teste,freq_abs)
Excel_Classificado


,Teste,Classificação
0,"@luau_drey hahahaha banco imobiliário, genius,...",1
1,nintendo pode anunciar controles flexíveis par...,1
2,rt @nintendeusa: os consoles da nintendo em ge...,0
3,sonhei com um super nintendo ???????????????????,1
4,@pedretweets eu sempre tive um amor enorme pel...,1
5,megaquarium daté sur nintendo switch https://t...,1
6,@katgatito amo todos menos os gordao jgoador d...,1
7,@ceythian @nintendo_mg miojão com suco de uva ...,1
8,@luizns6 @fabiocetf @mrbeambr cenário tá pront...,1
9,rt @nintendeusa: achei justa a escolha do terr...,2


In [181]:
classificador = []
for classficacao in Excel_Classificado["Classificação"]:
    classificador.append(classficacao)

print(classificador)

[1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 

In [199]:
Teste_de_Performance = {"Total":{"Certo":0,"Errado":0},0:{"Certo":0,"Errado":0},1:{"Certo":0,"Errado":0},2:{"Certo":0,"Errado":0},3:{"Certo":0,"Errado":0}}

for i in range(len(manual)):
    if classificador[i] == manual[i]:
        Teste_de_Performance[classificador[i]]["Certo"]+=1
        Teste_de_Performance["Total"]["Certo"]+=1
    else:
        Teste_de_Performance[classificador[i]]["Errado"]+=1
        Teste_de_Performance["Total"]["Errado"]+=1
            
    
    
print(Teste_de_Performance)
Porcentagem_de_Acerto ={}

for categoria in Teste_de_Performance:
    if Teste_de_Performance[categoria]["Certo"]!=0 and Teste_de_Performance[categoria]["Certo"]!=0:
        Porcentagem_de_Acerto[categoria] = Teste_de_Performance[categoria]["Certo"]/(Teste_de_Performance[categoria]["Certo"]+Teste_de_Performance[categoria]["Errado"])
print(Porcentagem_de_Acerto)


{'Total': {'Certo': 249, 'Errado': 151}, 0: {'Certo': 2, 'Errado': 3}, 1: {'Certo': 234, 'Errado': 141}, 2: {'Certo': 13, 'Errado': 7}, 3: {'Certo': 0, 'Errado': 0}}
{'Total': 0.6225, 0: 0.4, 1: 0.624, 2: 0.65}


___
### Concluindo

### Porque não podemos utilizar o próprio classificador para gerar mais amostras de treinamento?
Essa é uma pergunta que pode surgir quando se está fazendo um classificador, e apesar disso parecer uma boa ideia, esse método pode estragar seu classificador, uma vez que o intuito de gerar mais amostra de treinamento é justamente melhorar o classificador, uma vez que ele não é perfeito e pode cometer erros.

Se o classificador for utilizado para gerar mais amostras, ele vai acabar repetindo os seus erros mais comuns e ficar cada vez mais longe de ser um bom classificador, o processo de geração de amostras de treinamento deve ser feito manualmente, assim deixando o classificador cada vez mais eficiente

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**